In [ ]:
import requests
import json
import os
import pandas as pd
import time
from multiprocessing import Pool
from functools import partial
from time import sleep
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
'''
below are three functions, 1 gets a list of pdb ids that you derive by using the get_api function from the rcsb website.
so you go there and do an advance search to get what you want then you click the get api button to generate an api json query.
That is where the query variable I have here comes from.

The others download the pdbs from that query which should be chunked for multiprocessing
and the final one creates a dataframe of ncbi taxa info for the pdb ids for the purposes of analysis.

I used this to search for bacterial pdb ids that have 1 or more dsbs (around 10035).
I then got their taxa info which I use to cross against the MauE only catagories and the none catagories from the final set
of species (from the 35 phylum listed in the figures and datatables 8 and 8b ).
The pdbs in this list may then be downloaded as structures or their sequences retrieved for doing things like clustering or
predicting topology via topcons. This meant things had to have a sequence entry which they did if they had a uniprot ID.

I used topcons to remove things not exported to the periplasm, and I also removed entities without a proper uniprot ID. Many 
genes had multiple pdb entry structures that were redundant, so I examined them in the excell datatable 

These datatables are serperated into a folder to keep things sorta organized
'''

In [ ]:
#this was the query used to get the PDB_ids for entry with 1 or more disulfide bonds and in bacteria
#change 'false' to 'False' if remaking
query = {
  "query": {
    "type": "group",
    "logical_operator": "and",
    "nodes": [
      {
        "type": "terminal",
        "service": "text",
        "parameters": {
          "attribute": "rcsb_entry_info.disulfide_bond_count",
          "operator": "greater_or_equal",
          "negation": False, #this was changed to False from false and then the query worked 
          "value": 1
        }
      },
      {
        "type": "terminal",
        "service": "text",
        "parameters": {
          "attribute": "rcsb_entity_source_organism.ncbi_parent_scientific_name",
          "value": "Bacteria",
          "operator": "exact_match"
        }
      }
    ],
    "label": "text"
  },
  "return_type": "entry",
  "request_options": {
    "paginate": {
      "start": 0,
      "rows": 25
    },
    "results_content_type": [
      "experimental"
    ],
    "sort": [
      {
        "sort_by": "score",
        "direction": "desc"
      }
    ],
    "scoring_strategy": "combined"
  }
}

In [ ]:
#this is really fast
def pdb_id_getter(total_entries, query, divisor):
    total = total_entries
    chunks = divisor
    chunk_size = total // chunks                     
    num_remain = total - chunks * chunk_size  #this chunks the returned ids because there is a maxout I encountered      

    url = "https://search.rcsb.org/rcsbsearch/v2/query"

    data_total = []
    for i in range(chunks):
        start = i * chunk_size                       
        q = dict(query)                              
        q.setdefault("request_options", {})
        q["request_options"]["paginate"] = {"start": start, "rows": chunk_size}

        r = requests.post(url, json=q, timeout=60)
        r.raise_for_status()
        data_total.extend(r.json().get("result_set", []))

   
    if num_remain > 0:
        start = chunks * chunk_size                  
        q = dict(query)
        q.setdefault("request_options", {})
        q["request_options"]["paginate"] = {"start": start, "rows": num_remain}

        r = requests.post(url, json=q, timeout=60)
        r.raise_for_status()
        data_total.extend(r.json().get("result_set", []))

   
    pdb_ids = [item["identifier"] for item in data_total if "identifier" in item]
    return pdb_ids




In [ ]:
#this is really slow and should be done in parallel
def pdb_downloader(out_path,pdb_ids):
    pdb_list = pdb_ids
    out = out_path
    for pdb_id in pdb_ids:
        pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
        response = requests.get(pdb_url)
        with open(f"{out}/{pdb_id}.pdb", "wb") as f:
            f.write(response.content)
            f.close()
        print(f"Downloaded {pdb_id}.pdb")


In [ ]:
#bellow is the parallel process stuff that makes the pdb dataset construction faster
#the parallel funciton is general so you can run anything in this notebook as such


In [ ]:
ids_test = '1A0D'

def one_pdb_tax_uni_id_getter(pdb_ids, timeout = 60):
    ret_df = pd.DataFrame()
    ids_search = [pdb_ids]
    time = timeout
    taxids = []
    uniprot_ids = []
    sci_names = []
    retry_limit = 3  

    current = 0

    # graphql query from the graphql data api tool from rcsb. I then copied that to here
    query = """
        query annotations($id: String!) {
          entry(entry_id: $id) {
            polymer_entities {
              rcsb_entity_source_organism {
                ncbi_taxonomy_id
                common_name
                scientific_name
                ncbi_scientific_name
              }
              uniprots {
                rcsb_id
                rcsb_uniprot_protein {
                  source_organism {
                    scientific_name
                  }
                }
              }
            }
          }
        }
    """

    for i in ids_search:
        attempts = 0
        tax_id = None
        sci_name = None
        uniprot_id = None

        while attempts < retry_limit:
            try:
                json1 = {
                    "query": query,
                    "variables": {"id": i}
                }

                r = requests.post("https://data.rcsb.org/graphql", json=json1, timeout=time)
                r.raise_for_status()
                dict_temp = r.json()
                #print(dict_temp)  

        
                if "errors" in dict_temp and dict_temp["errors"]:
                    raise RuntimeError(dict_temp["errors"])

                polys = dict_temp['data']['entry']['polymer_entities']

                
                found = False
                for ent in polys:
                    src_org = ent.get('rcsb_entity_source_organism', [])
                    unis = ent.get('uniprots', [])
                    if src_org and unis:
                        tax_id = src_org[0].get('ncbi_taxonomy_id')
                        sci_name = src_org[0].get('ncbi_scientific_name') or src_org[0].get('scientific_name')
                        uniprot_id = unis[0].get('rcsb_id')
                        found = True
                        break

                if not found:
                    raise ValueError(f"No polymer entity with both taxonomy and UniProt for {i}")

                
                taxids.append(tax_id)
                sci_names.append(sci_name)
                uniprot_ids.append(uniprot_id)
                current += 1
                #print(current) 
                break  

            except Exception as e:
                attempts += 1
                print(f"Error for {i} (attempt {attempts}/{retry_limit}): {e}")
                if attempts < retry_limit:
                    sleep(10)

        if tax_id is None and sci_name is None and uniprot_id is None:
            taxids.append(None)
            sci_names.append(None)
            uniprot_ids.append(None)
            current += 1
            print(f"{i} failed after {retry_limit} attempts")
            print(current)


    return {
        "PDB_IDS": ids_search[0],
        "UP_Accessions": uniprot_id,
        "sci_names": sci_name,
        "species": tax_id,
    }

testdic = one_pdb_tax_uni_id_getter(ids_test, 60)
print(testdic)
print('----------\n----------')

###################

def fetch_seq_phyl_for_one_pid(pid, tax_filter=None, sleep=0.0):
    pid = pid
    url = (
        "https://rest.uniprot.org/uniprotkb/search?"
        f"query=(xref:pdb-{pid}%20OR%20xref:pdb-{pid})%20AND%20taxonomy_id:{tax_filter}"
        "&fields=accession,xref_pdb,lineage_ids,sequence&format=tsv"
    ) 
    if sleep > 0:
        time.sleep(sleep)

    for attempt in range(3):
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            break
        except Exception:
            if attempt == 2:
                return {
                    "PDB_IDS": pid,
                    "sequence": 0,
                    "phylum": 0,
                }
            time.sleep(1)

    lines = r.text.strip().split("\n")

    up_accessions = []
    sequence_for_pid = 0
    species_for_pid = 0

    if len(lines) > 1:
        for row in lines[1:]:
            parsed_row = row.split("\t")
            xref_str = parsed_row[1] #if you change the URL metadata order this will break. or if you add to it
            lineage = parsed_row[2] #if you change the URL metadata order this will break. or if you add to it
            lineages = lineage.split(',')
            
            for i in lineages:
                if 'phylum' in i:
                    phyl = i.split('(')[0] 
                    phylum = int(phyl)
                    break
                elif 'phylum' not in i:
                    phylum = None
                    #print('none_1')

            for ref in xref_str.split(";"):
                ref = ref.strip().upper()
                
                if ref == pid.upper():
                    sequence_for_pid = parsed_row[3] #if you change the URL metadata order this will break. or if you add to it
                    
    elif len(lines) <= 1:
        phylum = None
        sequence_for_pid = None
    return {
        "PDB_IDS": pid,
        "sequence": sequence_for_pid,
        "phylum": phylum,
    }

testdic2 = fetch_seq_phyl_for_one_pid(ids_test, tax_filter=2, sleep=0.05)
print(testdic2)
print('----------\n----------')

###############################

def run_parallel(func, items, max_workers=20, *args, **kwargs):
    rows = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(func, item, *args, **kwargs): item
            for item in items
        }

        for fut in as_completed(futures):
            rows.append(fut.result())

    return pd.DataFrame(rows)


'''
These test the speed of the functions. it shoudl be somewhere around 2-3 pdb IDs a second depending on how you set the sleep
times or how many threads you have available.
'''
start2 = time.time()
test_parallel = ['8TT3', '8TTF', '8TTH', '8TUM', '8TUN', '8TUW', '8TUZ', '8TV3', '8TV8', '8TV9', '8TVA', '8TVB', '8TVD', '8TVJ', '8TW4', '8TW6', '8TXO', '8TYC', '8TYE', '8TYZ', '8TZQ', '8U0Q', '8U1Y', '8U49', '8U4A', '8U4F', '8U4V', '8U4X', '8U59', '8U5B', '8U65', '8U6X', '8U79', '8U7I', '8U8Z', '8UAV', '8UAW', '8UBQ', '8UEN', '8UF9', '8UHF', '8UIW', '8UO7', '8UP2', '8UPI', '8UPO', '8UPR', '8UQL', '8UQM', '8UQP', '8UR0', '8URA', '8URH', '8URI', '8URN', '8URO', '8URX', '8URY', '8USM', '8UT1', '8UTA', '8UTB', '8UTL', '8UUB', '8UUP', '8UVZ', '8UY6', '8V3J', '8V4W', '8V7P', '8V8W', '8V93', '8V9G', '8V9H', '8V9V', '8VAS', '8VBA', '8VBB', '8VE9', '8VEK', '8VEL', '8VEM']
start = time.time()
results = run_parallel(fetch_seq_phyl_for_one_pid,
                       test_parallel,
                       tax_filter=2,
                       sleep=0.05,max_workers=50   
)
end = time.time()
elapsed = end-start

print(f'testing seq fetcher, elapsed {elapsed:.2f} seconds, ~{len(test_parallel)/elapsed:.2f} ids/sec')
print('----------\n----------')


test_parallel = ['8TT3', '8TTF', '8TTH', '8TUM', '8TUN', '8TUW', '8TUZ', '8TV3', '8TV8', '8TV9', '8TVA', '8TVB', '8TVD', '8TVJ', '8TW4', '8TW6', '8TXO', '8TYC', '8TYE', '8TYZ', '8TZQ', '8U0Q', '8U1Y', '8U49', '8U4A', '8U4F', '8U4V', '8U4X', '8U59', '8U5B', '8U65', '8U6X', '8U79', '8U7I', '8U8Z', '8UAV', '8UAW', '8UBQ', '8UEN', '8UF9', '8UHF', '8UIW', '8UO7', '8UP2', '8UPI', '8UPO', '8UPR', '8UQL', '8UQM', '8UQP', '8UR0', '8URA', '8URH', '8URI', '8URN', '8URO', '8URX', '8URY', '8USM', '8UT1', '8UTA', '8UTB', '8UTL', '8UUB', '8UUP', '8UVZ', '8UY6', '8V3J', '8V4W', '8V7P', '8V8W', '8V93', '8V9G', '8V9H', '8V9V', '8VAS', '8VBA', '8VBB', '8VE9', '8VEK', '8VEL', '8VEM']
start = time.time()
results2 = run_parallel(one_pdb_tax_uni_id_getter,
                       test_parallel,
                       timeout = 60,
                       max_workers=50   
)
end = time.time()
end2 = time.time()

elapsed = end-start
print(f'testing PDB meta fetcher, elapsed {elapsed:.2f} seconds, ~{len(test_parallel)/elapsed:.2f} ids/sec')

test_df = pd.merge(results,results2, on='PDB_IDS', how='right' )
test_df.to_excel(r"C:\PATH\cleaned_functions_test.xlsx")
elapsed2 = end2-start2
print(f'total time is {elapsed2}, with {len(test_parallel)/elapsed2} ids/sec')


In [ ]:
pdb_ids = pdb_id_getter(10035,query,10)
print(pdb_ids,len(pdb_ids))


In [ ]:
#this is the actual data acquisition
start= time.time()
results = run_parallel(fetch_seq_phyl_for_one_pid,
                       pdb_ids,
                       tax_filter=2,
                       sleep=0.05,max_workers=50)
end = time.time()
elapsed = end-start

results2 = run_parallel(one_pdb_tax_uni_id_getter,
                       pdb_ids,
                       timeout = 60,
                       max_workers=50)

bact_dsbs_df = pd.merge(results,results2, on='PDB_IDS', how='right' )
bact_dsbs_df.to_excel(r"C:\PATH\10_bacterial_dsbs_wmetadata.xlsx")

end = time.time()
elapsed = end-start
print(f'exported total time is {elapsed}, with {len(test_parallel)/elapsed} ids/sec')

In [ ]:
#now I drop whats not in the final phylogenetic analysis and remove redundant entries with the same sequence 

In [ ]:
print(bact_dsbs_df.shape)

In [ ]:

print(bact_dsbs_df)

In [ ]:
df = pd.read_excel(r"C:\PATH\5_Filtered_FBDS_IPRcounts_byspecies_MAUA_names.xlsx")
phyl = pd.read_excel(r"C:\PATH\8b_MAUgenes_3cat_3mark_perc_no_can_10more_names.xlsx")

In [ ]:
MauE_only = df[(df['IPR009908']>0) & (df['IPR003752'] == 0) & (df['IPR012932'] == 0)]
None_only = df[(df['IPR009908']==0) & (df['IPR003752'] == 0) & (df['IPR012932'] == 0)]

In [ ]:
print(MauE_only.shape,None_only.shape)

In [ ]:
maue_none = pd.concat([MauE_only,None_only],ignore_index=True)
print(maue_none.shape)

In [ ]:
print(maue_none['phylum'].nunique())

In [ ]:
phyl_list = list(phyl['phylum'])
for i in maue_none.index:
    phylum = maue_none.at[i,'phylum']
    if phylum not in phyl_list:
        maue_none = maue_none.drop(i)
print(maue_none.shape)

In [ ]:
print(maue_none['phylum'].nunique())

In [ ]:
#this made a list of species that are in the final data analysis for filtering the pdb dataset

In [ ]:
species_list = list(maue_none['species'])
print(len(species_list))

In [ ]:
bact_dsbs_df2 = bact_dsbs_df.copy()
print(len(species_list))
for i in bact_dsbs_df2.index:
    taxa = bact_dsbs_df2.at[i,'species']
    if taxa not in species_list:
        bact_dsbs_df2 = bact_dsbs_df2.drop(i)


In [ ]:
print(bact_dsbs_df2.shape)

In [ ]:
print(bact_dsbs_df2['PDB_IDS'].nunique())

In [ ]:
bact_dsbs_df2.to_excel(r"C:\PATH\11_bacterial_dsbs_wmetadata_filtered.xlsx")

In [ ]:
'''
now at this point I sorted the dataframe in excell by species ID. I did this because I wanted to see myself how many multiple
entries there were for each species. At this point I saw how many redundant pdb entries there
were for the same uniprot ID so I took the first pdb instance from the excel sheet (that was sorted by species ID in EXCEL not
PANDAS !Important! because they sort tables differently. This was just my personal preference, I like the way excel sorts.)
for each uniprot ID. So if there were 10 pdbs for P01552 I would only keep the PDB for the first one. The topcons information 
later comes from the full sequence from the uniprot ID entry (because it retains signal peptides and such) so the choice of PDB
ID is arbitrary. But I am explaining this for reproducibility, and incase you want to repeat the data analysis on your own. If
you are not careful you could get different PDB IDs from our work, likely representing the same uniprot IDs
'''

In [ ]:
bact_dsbs_df2 = pd.read_excel(r"C:\Users\bgonzale24\Desktop\MauE\notebooks\notebooks_p1\PDB_stuff\11_bacterial_dsbs_wmetadata_filtered.xlsx")

In [ ]:
print(bact_dsbs_df2)

In [ ]:
bact_dsbs_df3 = bact_dsbs_df2.drop_duplicates(subset='UP_Accessions',keep='first')
bact_dsbs_df3.to_excel(r"C:\PATH\12_bacterial_dsbs_wmetadata_filtered_no_redun.xlsx")

In [ ]:
#I need to drop things without a sequence 
bact_dsbs_df3 = pd.read_excel(r"C:\PATH\12_bacterial_dsbs_wmetadata_filtered_no_redun.xlsx")

In [ ]:
bact_dsbs_df3 =  bact_dsbs_df3.fillna(0)
for i in bact_dsbs_df3.index:
    sequence = bact_dsbs_df3.at[i,'sequence']
    if sequence != 0:
        print('seq')
    elif sequence == 0:
        print('noseq')
        bact_dsbs_df3 = bact_dsbs_df3.drop(i)
bact_dsbs_df3.to_excel(r"C:\PATH\13_bacterial_dsbs_wmetadata_filtered_no_redun_wseqs.xlsx")

In [ ]:
#now I am going to export fastas for topcons

In [ ]:
for i in bact_dsbs_df3.index:
    acc = bact_dsbs_df3.at[i,'UP_Accessions']
    sequence = bact_dsbs_df3.at[i,'sequence']
    if sequence:
        string = f'>{acc}\n{sequence}\n'
        with open(r"C:\PATH" + f'\{acc}.fasta', 'w') as file:
            file.write(string)
            file.close()
    elif not sequence:
        print('noseq')
        

In [ ]:
#at this point topcons was done using a docker conatiner as per the topcons2 github 
#I will now import the topcons data
basedir = "C:/PATH/"
topcons_results=[]
for i in bact_dsbs_df3.index:
    acc = bact_dsbs_df3.at[i,'UP_Accessions']
    path = basedir + f"{acc}/query.result.txt.fa"
    with open(path,'r') as file:
        tc = file.readlines()[1]
        topcons_results.append(tc)
        file.close
bact_dsbs_df3['TOPCONS2'] = topcons_results

In [ ]:
bact_dsbs_df3.to_excel(r"C:\PATH\14_bacterial_dsbs_wmetadata_filtered_no_redun_wseqs_TC2.xlsx")

In [ ]:
#now I drop stuff without a SP or TM

In [ ]:
#dropping things without sp or TM
print(bact_dsbs_df3.shape)
for i in bact_dsbs_df3.index:
    tc = bact_dsbs_df3.at[i,'TOPCONS2']
    if 'S' in tc:
        continue
    elif 'M' in tc:
        continue
    else:
        bact_dsbs_df3 = bact_dsbs_df3.drop(i)
print(bact_dsbs_df3.shape)

In [ ]:
bact_dsbs_df3.to_excel(r"C:\PATH\15_bacterial_dsbs_for_manual_check.xlsx")

In [ ]:
'''
At this point I can only manually annotate the dataset. This is because the disulfide bonds may still be junk that slipped 
through (Like artificial or mutated) or they may be functional like a CXXC or CXC active site. To the best of our out ability
the group looked at the remaining 114 structures and annotated them as and pulled the genuine disulfide bond entries that 
we had a consensus on. These are found in data table 16. We then manually back checked if they were MauE only or None species
and annotated that as well. This was finally used to make the table in the publication using word to make them nice and neat.
There were a few duplicates that slipped by that I manually removed as well. 
'''